## Aim

Look at when and for how long actors and "Stephen Reicher" and "John Drury" appear in the filtered articles.

In [1]:
# looks like there is a Colin Drury in the data

In [2]:
import os

In [3]:
import numpy as np

In [4]:
from math import log2
from typing import Union

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
import networkx as nx
from operator import itemgetter

In [7]:
%matplotlib inline

In [8]:
from IPython.core.interactiveshell import InteractiveShell

In [9]:
from src.news_media.get_keywords_trend import *

/Users/alessiatosi/DS_projects/behavioural-sci-perception/docs/ext/keywords.yaml has been successfully loaded as a dict
/Users/alessiatosi/DS_projects/behavioural-sci-perception/docs/ext/subkw_to_kw_map.yaml has been successfully loaded as a dict


In [10]:
pd.set_option('display.max_colwidth', None)

In [11]:
DATA_DIR = os.environ.get("DIR_DATA_INTERIM")

## Import UK's news articles

In [13]:
news_uk = NewsArticles()

(1968, 13)
647
(1968, 13)
647
Index(['Unnamed: 0', 'article_id', 'title', 'pub_date', 'full_text',
       'subkword', 'opinion_context', 'opinion_context_id',
       'keyword_sentiment', 'refers_to_gov', 'gov_sentiment', 'kword',
       'pub_date_dt'],
      dtype='object')
647


`news_uk` is a `NewsArticles` class instance, with the following public attributes and methods:

In [14]:
[d for d in dir(news_uk) if not d.startswith("_")]

['allwords_raw_tf',
 'data_raw',
 'data_uniq_article',
 'get_kword_docfreq_agg',
 'get_kword_rawfreq',
 'get_kword_rawfreq_agg',
 'get_num_ngrams',
 'kword_docfreq_agg',
 'kword_rawfreq',
 'kword_rawfreq_agg',
 'kword_reldocfreq_agg',
 'kword_relfreq_agg',
 'kword_rfrdf_agg',
 'kword_yn_occurrence',
 'unigram_count_perdoc']

`news_uk.data` contains the original dataset of articles:

## Extract occurrences of "reicher" and "drury"

The actors who emerged during the qualitative research

In [ ]:
## Extract data needed for analysis

In [ ]:
# from src.news_media.extract_kword_sentence import collect_kword_opinioncontext


In [17]:
articles = [re.sub(r'([,.!?])([a-zA-Z])', r'\1 \2', article) for article in news_uk.data_uniq_article.full_text]

In [18]:
list_sentences = [tokenise_sent(article.lower()) for article in articles]

In [19]:
list_sentences[0]

['rush hour, york city centre, sometime around the end of the decade.',
 'stress-free commuters glide by on bicycles, community gardens are tended by volunteers as they leave work, and early evening diners spill outside on to table-filled squares.',
 'over in fossgate, a mid-week street festival is just beginning.',
 'at the station, a line of on-demand driverless pods whisk arriving rail passengers to their end destinations.',
 'such is what this north yorkshire city could look like if proposals approved by the council to make the centre car free by 2023 come to fruition.',
 'or so say advocates.',
 'others, including the chamber of commerce, are not entirely convinced.',
 'there are concerns that less vehicles could ultimately mean reduced footfall leading to a nosediving economy, struggling businesses and plummeting job opportunities.',
 'it is, in many ways, says one councillor, a fight for the heart and future of the city.',
 'yet this is not a battle which will be confined to yor

In [20]:
opinion_contexts = [[[keyw, (idx, sentence)] for idx, sentence in enumerate(sentences)
            for keyw in ['reicher', 'drury']
            if keyw in word_tokenize(sentence)] for sentences in list_sentences]


In [21]:
df = news_uk.data_uniq_article

In [22]:
df['kw_opinioncontext'] = opinion_contexts

In [23]:
df.columns

Index(['article_id', 'pub_date_dt', 'title', 'full_text', 'preproc_text',
       'word_count', 'kw_opinioncontext'],
      dtype='object')

In [24]:
df.head()

article_id pub_date_dt  \
10       b1_6  2020-01-27   
11       b1_7  2020-01-28   
12       b1_8  2020-01-29   
13       b1_9  2020-01-29   
15      b1_10  2020-02-03   

                                                                                                                                                                                                                             title  \
10                                              A car-free future: are cities moving to a pedestrian age?;With York announcing plans to ban vehicles from its centre, Colin Drury asks: is it a blueprint for a better urban life?   
11                                                        She can't say no': the Ugandan men demanding to be breastfed;A study is looking into the coercive practice in Uganda, amid calls for the government to address the issue   
12  Why we're falling out of love with Love Island;As interest in this series' group of bored-looking contestants dwindles, Roisin O'Connor wonders whether changing social attitudes to marriage and relationships is the problem   
13                                                                                                                                                                                        Phone app checks if students go to class   
15                                                                                                                                                                                         An equivocal book for a polarised world   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [25]:
# explode a list like column
df = df.explode('kw_opinioncontext')

In [26]:
# split keyword and context across two separate columns
    # way-around due to certain kw_opinioncontext = NaN
for i in [0, 1]:
    df[str(i)] = [k[i] if isinstance(k, list) else k for k in df.kw_opinioncontext]

In [27]:
df.rename(columns={'0': 'kword', '1': 'opinion_context'}, inplace=True)

In [28]:
df.columns

Index(['article_id', 'pub_date_dt', 'title', 'full_text', 'preproc_text',
       'word_count', 'kw_opinioncontext', 'kword', 'opinion_context'],
      dtype='object')

In [29]:
# filter where opinion_context is not empty 
df = df[df['kw_opinioncontext'].str.len() > 0].copy()

In [30]:
# add a unique id per opinion context
df['opinion_context_id'] = range(1, df.shape[0] + 1)

In [32]:
df.columns

Index(['article_id', 'pub_date_dt', 'title', 'full_text', 'preproc_text',
       'word_count', 'kw_opinioncontext', 'kword', 'opinion_context',
       'opinion_context_id'],
      dtype='object')

In [34]:
df[['article_id', 'title', 'pub_date_dt', 'kword', 'opinion_context']]

,article_id,title,pub_date_dt,kword,opinion_context
886,b1_355,"For us to listen, those giving orders need to share the pain",2020-04-13,reicher,"(13, psychologists alexander haslam and stephen reicher argued that when one looks closely at what actually happened during the milgram experiment, it was not issuing orders that achieved co-operation.)"
1162,b1_472,Revealed: the inside story of the UK's Covid-19 crisis;How herd immunity and delayed lockdown hampered efforts to contain the spread of coronavirus Coronavirus - latest updatesSee all our coronavirus coverage,2020-04-29,reicher,"(138, three behavioural scientists on spi-b, stephen reicher, john drury and clifford stott, even wrote an article for the psychologist journal, rejecting the notion of ""fatigue"" and suggesting that delaying stricter social distancing measures on that premise was taking a risk with lives.)"
1162,b1_472,Revealed: the inside story of the UK's Covid-19 crisis;How herd immunity and delayed lockdown hampered efforts to contain the spread of coronavirus Coronavirus - latest updatesSee all our coronavirus coverage,2020-04-29,drury,"(138, three behavioural scientists on spi-b, stephen reicher, john drury and clifford stott, even wrote an article for the psychologist journal, rejecting the notion of ""fatigue"" and suggesting that delaying stricter social distancing measures on that premise was taking a risk with lives.)"
1195,b1_499,"Isolation as dangerous as smoking 15 cigarettes a day, lockdown adviser warns;Prof Stephen Reicher says that social distancing is having an impact on the nation's mental and physical health",2020-05-02,reicher,"(3, professor stephen reicher, who is part of a group of behavioural scientists advising the government, told the telegraph: ""the problem with lockdown is isolation; being cut off from people is bad for you psychologically and physically.)"
1195,b1_499,"Isolation as dangerous as smoking 15 cigarettes a day, lockdown adviser warns;Prof Stephen Reicher says that social distancing is having an impact on the nation's mental and physical health",2020-05-02,reicher,"(15, prof reicher said that from the data he has seen the ""overwhelming story [of lockdown] is still how high the levels of adherence are and how positive relations between people are.)"
...,...,...,...,...,...
1956,b2_245.0,"Public baffled by victory messages, claims adviser",2020-05-14,reicher,"(1, steve reicher, professor of social psychology at the university of st andrews, and a member of spi-b, the government's advisory group on behaviour, said that people were now justifiably confused about coronavirus messages.)"
1956,b2_245.0,"Public baffled by victory messages, claims adviser",2020-05-14,reicher,"(9, professor reicher said that the sign of a good message was that people understood it, and that in this case polling clearly showed it had failed.)"
1956,b2_245.0,"Public baffled by victory messages, claims adviser",2020-05-14,reicher,"(12, professor reicher said that secrecy about the scientific advisory groups, including initially redacting some released documents, was also counterproductive.)"
1960,b2_248.0,"Heated arguments' at Sage over science behind easing lockdown, says senior adviser;Advisory group",2020-05-11,reicher,"(14, one member of the government's spi-b advisory committee, stephen reicher, a professor of social psychology at the university of st andrews, wrote on social media: ""the greatest asset we have in this crisis is the trust and adherence of the public.)"


In [35]:
# save
df[['article_id', 'title', 'pub_date_dt', 'kword', 'opinion_context']].to_csv(
    os.path.join(DATA_DIR, "emerging_actors.csv"))

### Calculate prominence metrics for emergent actors

In [ ]:
# Read in additional required data

In [47]:
DIR_DATA = os.environ.get("DIR_DATA_INTERIM")

In [48]:
DIR_DATA_EXTRA = os.environ.get("DIR_DATA_EXTRA")

In [49]:
# prominence
term_freqs_nm = "kword_rawfreq_2W-MON.csv"
doc_freqs_nm = "kword_docfreq_2W-MON.csv"
metrics_nm = "kword_rfrdf_2W-MON.csv"

In [50]:
doc_freqs = pd.read_csv(os.path.join(DIR_DATA, doc_freqs_nm))
term_freqs = pd.read_csv(os.path.join(DIR_DATA, term_freqs_nm))
metrics = pd.read_csv(os.path.join(DIR_DATA, metrics_nm))

In [ ]:
# Calculate reicher's and drury's fortnight number of occurrences

In [39]:
df.set_index('pub_date_dt', inplace=True)

In [77]:
emergent_actors_agg = df.groupby(['kword', pd.Grouper(freq="W-MON", closed="left", label="left")]).agg({'article_id':"count"})

In [78]:
emergent_actors_agg

article_id
kword   pub_date_dt            
drury   2020-04-27            1
        2020-05-04            2
        2020-05-11            1
        2020-05-25            5
        2020-06-08            2
reicher 2020-04-13            1
        2020-04-27            6
        2020-05-04            4
        2020-05-11            5
        2020-05-18           16
        2020-05-25           43
        2020-06-01            2
        2020-06-08            1
        2020-06-15            6
        2020-06-22           26
        2020-06-29            3

In [54]:
# save in wide format
emergent_actors_agg.reset_index(inplace=True)
emergent_actors_agg_w = pd.pivot(emergent_actors_agg, "pub_date_dt", "kword", "article_id")

In [76]:
emergent_actors_agg_w

kword,drury,reicher
pub_date_dt,,
2020-04-13,NaN,1.0
2020-04-27,3.0,10.0
2020-05-11,1.0,21.0
2020-05-25,5.0,45.0
2020-06-08,2.0,7.0
2020-06-22,NaN,29.0


In [69]:
from datetime import datetime

In [71]:
doc_freqs.fortnight_starting = [datetime.strptime(w, "%Y-%m-%d") for w in doc_freqs.fortnight_starting]

In [73]:
doc_freqs[['fortnight_starting', 'article_count']].merge(emergent_actors_agg_w, "left", 
                                                         left_on="fortnight_starting", right_on="pub_date_dt")

,fortnight_starting,article_count,drury,reicher
0,2020-01-27,10,NaN,NaN
1,2020-02-10,13,NaN,NaN
2,2020-02-24,26,NaN,NaN
3,2020-03-09,108,NaN,NaN
4,2020-03-23,62,NaN,NaN
5,2020-04-06,63,NaN,NaN
6,2020-04-20,71,NaN,NaN
7,2020-05-04,102,NaN,NaN
8,2020-05-18,83,NaN,NaN
9,2020-06-01,39,NaN,NaN


In [79]:
doc_freqs

,fortnight_starting,american_behav_scientists,behav_analysis,behav_change,behav_econ,behav_insight,behav_insights_team,behav_science,behav_scientist,behavioural_economist,behavioural_fatigue,chater,halpern,herd_immunity,michie,nudge,psychologist,psychology,spi-b,article_count
0,2020-01-27,1,0,0,1,0,1,0,3,0,0,0,0,0,0,5,0,1,0,10
1,2020-02-10,2,0,0,1,1,1,3,1,0,0,0,0,0,0,4,2,0,0,13
2,2020-02-24,4,1,0,1,0,6,14,1,0,0,0,0,0,0,3,3,2,0,26
3,2020-03-09,5,0,3,3,1,30,56,7,1,3,1,24,54,12,14,5,24,2,108
4,2020-03-23,3,1,11,6,2,8,32,6,0,2,0,10,16,12,12,5,8,2,62
5,2020-04-06,4,2,3,6,2,5,35,9,0,3,0,1,14,5,6,7,10,3,63
6,2020-04-20,5,0,4,2,3,9,37,8,2,1,1,2,9,17,8,9,19,9,71
7,2020-05-04,3,4,8,4,0,11,34,10,3,0,0,6,8,21,2,8,20,25,102
8,2020-05-18,2,1,3,5,5,3,47,8,2,0,0,3,9,15,3,7,18,22,83
9,2020-06-01,0,0,4,0,0,1,22,2,0,0,0,0,3,11,1,4,7,9,39


In [75]:
emergent_actors_agg_w

kword,drury,reicher
pub_date_dt,,
2020-04-13,NaN,1.0
2020-04-27,3.0,10.0
2020-05-11,1.0,21.0
2020-05-25,5.0,45.0
2020-06-08,2.0,7.0
2020-06-22,NaN,29.0
